In [128]:
import openai
from transformers import GPT2TokenizerFast

from datasets import load_dataset

import yaml
import tqdm

In [134]:

DATASET_NAME = "cnn_dailymail"
DATASET_CONFIG = "3.0.0"

N_TRAIN_SAMPLES = 5
N_VAL_SAMPLES = 5
N_TEST_SAMPLES = 5

MAX_TOKENS_RESPONSE = 4096

MODEL_MAX_TOKENS = 2048

OPENAI_CONFIG = {
    "engine": "text-davinci-003", # text-curie-001
    "temperature": 1,
    "n": 1,
    "echo": False,
    }


In [91]:
dataset = load_dataset(DATASET_NAME, DATASET_CONFIG)

Found cached dataset cnn_dailymail (C:/Users/navjo/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00,  8.14it/s]


In [49]:
with open("keys.yaml", "r") as f:
    openai.api_key = yaml.load(f, Loader = yaml.loader.SafeLoader)["openai_key"]

In [120]:
completion = openai.Completion.create(**OPENAI_CONFIG, prompt="Hello world")

In [122]:
completion

<OpenAIObject text_completion id=cmpl-6cuaukW4ABak08NUTi1vBeJIjRIhc at 0x1dbe3c40048> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nHello world! This is the world's simplest text editor."
    }
  ],
  "created": 1674732032,
  "id": "cmpl-6cuaukW4ABak08NUTi1vBeJIjRIhc",
  "model": "text-curie-001",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 2,
    "total_tokens": 16
  }
}

In [3]:
gpt_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [99]:
def generate_prompt(article):
    return f"""Oversett den følgende nyhetsartikkelen til Norsk:: 
{article}
    
Oversatt til Norsk:"""

def get_prompt_tokens(article, tokenizer):
    tokenized_article = gpt_tokenizer(article)['input_ids']
    return len(tokenized_article)

In [96]:
for model in engines["data"]:
    print(model)

{
  "created": null,
  "id": "babbage",
  "object": "engine",
  "owner": "openai",
  "permissions": null,
  "ready": true
}
{
  "created": null,
  "id": "ada",
  "object": "engine",
  "owner": "openai",
  "permissions": null,
  "ready": true
}
{
  "created": null,
  "id": "davinci",
  "object": "engine",
  "owner": "openai",
  "permissions": null,
  "ready": true
}
{
  "created": null,
  "id": "text-embedding-ada-002",
  "object": "engine",
  "owner": "openai-internal",
  "permissions": null,
  "ready": true
}
{
  "created": null,
  "id": "babbage-code-search-code",
  "object": "engine",
  "owner": "openai-dev",
  "permissions": null,
  "ready": true
}
{
  "created": null,
  "id": "text-similarity-babbage-001",
  "object": "engine",
  "owner": "openai-dev",
  "permissions": null,
  "ready": true
}
{
  "created": null,
  "id": "text-davinci-001",
  "object": "engine",
  "owner": "openai",
  "permissions": null,
  "ready": true
}
{
  "created": null,
  "id": "curie-instruct-beta",
  "obj

Kjente ting vi kan filtrere ut:

"E-mail to a friend"
Alt etter første " -- " 

In [171]:
dataset["train"][0]["highlights"]

"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund ."

In [189]:
def translate_article(article):

    # Fjerner London, England (Reuters) --
    article = " ".join(article.split(" -- ")[1:])

    # Fjerner alt etter E-mail to a friend
    article = article.split("E-mail to a friend")[0]

    # article
    full_article_prompt = generate_prompt(article)

    # Number of tokens in article prompt
    article_prompt_tokens = get_prompt_tokens(full_article_prompt, gpt_tokenizer)

    # Prøver å begrense lengden noe
    article_translated = openai.Completion.create(**OPENAI_CONFIG, prompt=full_article_prompt, max_tokens=MODEL_MAX_TOKENS-article_prompt_tokens-25)["choices"][0].text.strip()

    return article_translated

def translate_highlight(highlight):

    full_highlight_prompt = generate_prompt(highlight)

    highlight_prompt_tokens = get_prompt_tokens(full_highlight_prompt, gpt_tokenizer)

    highlight_translated = openai.Completion.create(**OPENAI_CONFIG, prompt=full_highlight_prompt, max_tokens=MODEL_MAX_TOKENS-highlight_prompt_tokens-25)["choices"][0].text.strip()
    return highlight_translated

In [198]:
n_translated = 0

translated_dataset = {"article": [], "highlights": [], "original_article": [], "original_highlights": []}


for sample in (dataset["train"]):
    print("Sample", n_translated)
    if n_translated < N_TRAIN_SAMPLES:

        article_translated = translate_article(sample["article"])

        highlight_translated = translate_highlight(sample["highlights"])
        
        translated_dataset["article"].append(article_translated)
        translated_dataset["highlights"].append(highlight_translated)

        translated_dataset["original_article"].append(sample["article"])
        translated_dataset["original_highlights"].append(sample["highlights"])

        n_translated += 1

    else:
        break


Sample 0
Sample 1
Sample 2
Sample 3
Sample 4
Sample 5


In [199]:
translated_dataset

{'article': ['Harry Potter-stjernen Daniel Radcliffe får tilgang til en rapportert £20 millioner ($41,1 millioner) formue når han fyller 18 år på mandag, men han insisterer på at pengene ikke vil øse over ham. Til skuffelse for sladderblad rundt om i verden, sier den unge skuespilleren at han ikke har noen planer om å bruke bort pengene sine på hurtigbiler, drikke og kjendis-fester. «Jeg planlegger ikke å være én av de som, så snart de fyller 18, plutselig kjøper seg en enorm samling sportsbiler eller noe lignende», sa han til en australsk intervjuer tidligere denne måneden. «Jeg tror ikke jeg kommer til å være spesielt utsvevende. «De tingene jeg liker å kjøpe, koster under 10 pund - bøker, CDer og DVDer.» På 18, vil Radcliffe være i stand til å spille på casino, kjøpe en drink i en pub eller se skrekkfilmen "Hostel: Part II," for tiden på seksende plass på den britiske bokse-kontoren chart. Detaljer om hvordan han vil markere sin største bursdag er i tillegg. Hans agent og publiserin